In [0]:
# layer1
# -----------
# KNN
# LR
# RF
# XGB 
# NN

# layer2
# -----------
# XGB
# NN

# -----------
# avg(XGB+NN)


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
import pickle
import gc
# import statsmodels.api as sm
from scipy import stats

from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV,GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression,Ridge,Lasso
from sklearn.metrics import *
from sklearn.feature_selection import RFE, RFECV,VarianceThreshold, chi2
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, Normalizer 

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
# import catboost as cb
# from catboost import Pool, CatBoostClassifier

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l1_l2

Using TensorFlow backend.


## Helpers

`Ctrl+ Shift + i` to `open` inspector view . Then goto console.

```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```

In [2]:
# Mount Gdrive method-1
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# f1_metric
from sklearn.metrics import f1_score, roc_auc_score
# this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483


def f1_metric(y_hat, y_true):
    y_true = y_true.get_label()
    preds = y_hat.reshape(4, -1).T
    preds = preds.argmax(axis = 1)
    return 'f1', f1_score(y_true, preds,  average='weighted'), True

def auc_metric(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)       # scikits f1 doesn't like probabilities
    return 'f1', roc_auc_score(y_true, y_hat), True



In [0]:
# LightGBM f1 metric

def multi_class_f1_score_factory(num_classes, average):
    """Factory for LightGBM multi class F1-score function.
    Parameters
    ----------
    num_classes : int
        Number of classes to classify.
    average : string, 'micro' or 'macro'
        ``'micro'``
            Calculate metrics globally by counting the total true positives,
            false negatives and false positives.
        ``'macro'``
            Calculate metrics for each label, and find their unweighted mean.
            This does not take label imbalance into account.
    """
    if average != 'macro' and average != 'micro':
        raise ValueError("average should be 'macro' or 'micro'")

    eval_name = 'f1_' + average

    def multi_class_f1_score(y_pred, data):
        y_true = data.get_label()
        y_pred = y_pred.reshape((num_classes, -1))
        y_pred = np.transpose(y_pred)
        y_pred = np.argmax(y_pred, axis=1)
        return eval_name, f1_score(y_true, y_pred, average=average), True

    return multi_class_f1_score


# feval=multi_class_f1_score_factory(num_classes, 'macro')

In [0]:
def count_feature(X, tbl_lst = None, min_cnt = 1):
    X_lst = [pd.Series(X[:, i]) for i in range(X.shape[1])]
    if tbl_lst is None:
        tbl_lst = [x.value_counts() for x in X_lst]
        if min_cnt > 1:
            tbl_lst = [s[s >= min_cnt] for s in tbl_lst]
    X = sp.column_stack([x.map(tbl).values for x, tbl in zip(X_lst, tbl_lst)])
    # NA(unseen values) to 0
    return np.nan_to_num(X), tbl_lst

def elapsed_time(start_time, end_time):
    elapsed_sec = end_time - start_time
    h = int(elapsed_sec / (60 * 60))
    m = int((elapsed_sec % (60 * 60)) / 60)
    s = int(elapsed_sec % 60)
    return "{}:{:>02}:{:>02}".format(h, m, s)

## Stacking functions

In [0]:
def standarize_feature(train_df, test_df, cols):
    scaler = StandardScaler()
    for col in cols:
        train_df[col] = scaler.fit_transform(train_df[col].values.reshape(-1,1).astype(np.float32))
        test_df[col] = scaler.transform(test_df[col].values.reshape(-1,1).astype(np.float32))
    return None

def extend_bounds(bins):
    bins[0] = bins[0] - 1
    bins[-1] = bins[-1] + 1


def discretize_feature(train_df, test_df, cols, num_bins, how='equal_freq'):
    if how == 'equal_width':
        for col in cols:
            group_names = range(num_bins)
            train_df[col], bins = pd.cut(train_df[col], num_bins, labels=group_names, retbins=True)
            test_df[col] = pd.cut(test_df[col], bins, labels=group_names)
    elif how == 'equal_freq':
        for col in cols:
            freq_per_bin = train_df[col].shape[0] / num_bins
            values = sorted(train_df[col])
            bins = [values[0]]
            for i in range(1, num_bins + 1):
                if i < num_bins:
                    bins.append(values[i * freq_per_bin - 1])
                else:
                    bins.append(values[-1])
            group_names = range(num_bins)

            i = 1
            while i < len(bins):
                if bins[i] == bins[i-1]:
                    del bins[i]
                    del group_names[i]
                else:
                    i += 1
            extend_bounds(bins)
            train_df[col] = pd.cut(train_df[col], bins, labels=group_names)
            test_df[col] = pd.cut(test_df[col], bins, labels=group_names)
    return None

In [0]:
def process_data(train_df, test_df, ylabel='target', numerical_features=None, standarization=False, discretization=False, transform=None):
    # numerical_features = train_df.columns

    numerical_features = numerical_features
    if standarization:
        standarized_features = numerical_features
        standarize_feature(train_df, test_df, standarized_features)
        
    if discretization:
        discretized_features = numerical_features
        discretize_feature(train_df, test_df, discretized_features, num_bins=10, how='equal_freq')
    
    X = train_df.drop(ylabel, axis=1).as_matrix()
    y = train_df[ylabel].as_matrix()
    X_submission = test_df.as_matrix()
    
    if transform == 'log':
        X = np.log1p(X)
        X_submission = np.log1p(X_submission)
    elif transform == 'sqrt':
        X = np.sqrt(X + 3.0 / 8)
        X_submission = np.sqrt(X_submission + 3.0 / 8)
    elif transform == 'pca':
        pca = PCA(n_components=3).fit(X)
        X = pca.transform(X)
        X_submission = pca.transform(X_submission)
    elif transform == 'tsne':
        tsne = TSNE(n_components=3).fit(X)
        X = tsne.transform(X)
        X_submission = tsne.transform(X_submission)
    elif transform == 'pca+':
        pca = PCA(n_components=3).fit(X)
        X = np.hstack((X, pca.transform(X)))
        X_submission = np.hstack((X, pca.transform(X)))
    elif transform == 'tsne+':
        tsne = TSNE(n_components=3).fit(X)
        X = np.hstack((X, tsne.transform(X)))
        X_submission = np.hstack((X_submission, tsne.transform(X_submission)))        
    return X, y, X_submission

In [0]:
def evaluate(y, y_pred):
    mse = mean_squared_error(y, y_pred)
    return mse

In [0]:
def models_CV_train(models, X, y, X_submission, n_classes=1, n_folds=5):
    summary = {}

    kf = list(KFold(n_folds, random_state=42).split(X, y))
    
    stack_train = np.zeros((X.shape[0], n_classes, len(models)))
    stack_test = np.zeros((X_submission.shape[0], n_classes, len(models)))
    
    for i, model in enumerate(models):
        print("Model %d:" % i, model)
        
        avg_logloss = 0
        
        stack_test_model_i = np.zeros((X_submission.shape[0], n_classes, len(skf)))
        for j, (train_idx, test_idx) in enumerate(skf):
            print("  Fold %d" % j)
            X_train = X[train_idx]
            y_train = y[train_idx]
            X_test = X[test_idx]
            y_test = y[test_idx]

            # if type(model) != dict:
            #     model.fit(X_train, y_train)
            #     y_test_pred = model.predict_proba(X_test)  
            #     y_submission_pred = model.predict_proba(X_submission)   

            # else:
            #     print('Fit LightGBM model with early stopping.. \n')
            #     d_train = lgb.Dataset(X_train, y_train, categorical_feature=[7])  #categ_feats
            #     d_eval = lgb.Dataset(X_test, y_test, categorical_feature=[6])     # categ_feats
                
            #     model = lgb.train(lgb_params, d_train, num_boost_round=1500, 
            #                        valid_sets=(d_train, d_eval), 
            #                        early_stopping_rounds=50,
            #                        verbose_eval=200,
            #                        feval=f1_metric,
            #                        evals_result={} )

            #     y_test_pred = model.predict(X_test)  #num_rounds=model.best_
            #     y_submission_pred = model.predict(X_submission)      
            
            # elif type(model) == XGBClassifier:    # xgboost.sklearn.
            #     print('Fit XGBOOST model with early stopping.. \n')
            #     d_train = xgb.DMatrix(X_train, y_train)  
            #     d_test = xgb.DMatrix(X_test, y_test)     
                
            #     model = xgb.train(xgb_params, dtrain, num_boost_round=30, 
            #                       evals=[(d_train,'train'), (d_test,'eval')], obj=None,
            #                       early_stopping_rounds=30, verbose_eval=200)
                
            #     y_test_pred = model.predict_proba(X_test)   # best_ntree_limit

            model.fit(X_train, y_train)
            y_test_pred = model.predict_proba(X_test)  

            stack_train[test_idx, :, i] = y_test_pred
            
            score = evaluate(y_test, y_test_pred)
            avg_score += score
            print("logloss: %f" % score)
            
            y_submission_pred = model.predict_proba(X_submission)           
            stack_test_model_i[:, :, j] = y_submission_pred
        
        avg_score = avg_score / n_folds
        print("model average logloss: %f" % avg_score)
        summary[i] = avg_score
        
        stack_test[:, :, i] = stack_test_model_i.mean(axis=2)

    return np.swapaxes(stack_train, 1, 2).reshape((X.shape[0], -1)), np.swapaxes(stack_test, 1, 2).reshape((X_submission.shape[0], -1)), summary



# rscv = RandomizedSearchCV(estimator=lgb_model, 
#                            param_distributions=hyperparams, 
#                            n_iter=30, 
#                            cv=kf, 
#                            scoring=SCORE,   
#                            random_state=SEED, verbose=1, n_jobs=-1)

# rscv.fit(data_train[feats], data_train[target])



In [0]:
# d_train = lgb.Dataset(X_train, y_train, categorical_feature=[6])  #categ_feats
# d_eval = lgb.Dataset(X_test, y_test, categorical_feature=[6])     # categ_feats
                
# lgb.train(lgb_params, d_train, num_boost_round=1500, 
#                                    valid_sets=(d_train, d_eval), 
#                                    early_stopping_rounds=50,
#                                    verbose_eval=200,
#                                    feval=f1_metric,
#                                    evals_result={} )

In [0]:
def stacking_models_CV_train(models, X, y, X_submission, n_classes=1, n_folds=5):

    kf = list(KFold(n_folds, random_state=42).split(X, y))
    
    y_submission_pred = np.zeros((X_submission.shape[0], n_classes, len(models)))
        
    for i, model in enumerate(models):
        print("Stacking Model %d" % i, model)
        avg_logloss = 0
        y_submission_pred_model_i = np.zeros((X_submission.shape[0], n_classes, len(kf)))
        for j, (train_idx, test_idx) in enumerate(skf):
            print("Fold", j)
            X_train = X[train_idx]
            y_train = y[train_idx]
            X_test = X[test_idx]
            y_test = y[test_idx]
            
            # model.fit(X_train, y_train)
            
            if type(model) == XGBClassifier:    
                print('Fit XGBOOST model with early stopping.. \n')
                d_train = xgb.DMatrix(X_train, y_train)  
                d_test = xgb.DMatrix(X_test, y_test)     
                
                model = xgb.train(xgb_params, dtrain, num_boost_round=30, 
                                  evals=[(d_train,'train'), (d_test,'eval')], obj=None,
                                  early_stopping_rounds=30, verbose_eval=200)
                
            y_test_pred = model.predict_proba(X_test)   # best_ntree_limit
            
            y_submission_pred_model_i[:, :, j] = model.predict_proba(X_submission)
            
            score = evaluate(y_test, y_test_pred)
            avg_score += score
            print("  logloss: %f" % score)

        avg_score = avg_score / n_folds
        print("model average logloss: %f" % avg_score)

        y_submission_pred[:, :, i] = y_submission_pred_model_i.mean(axis=2)

    return y_submission_pred  



In [0]:
def create_2_layer_keras_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dropout(0.05, input_shape=(input_dim,)))
    model.add(Dense(512, init='glorot_normal', activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    model.add(Dropout(0.5))
    model.add(Dense(256, init='glorot_normal', activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    model.add(Dropout(0.5))
    model.add(Dense(output_dim, init='glorot_normal', activation='softmax', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])    # 'categorical_crossentropy'
    return model


def create_3_layer_keras_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dropout(0.05, input_shape=(input_dim,)))
    model.add(Dense(1024, init='glorot_normal', activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    model.add(Dropout(0.5))    
    model.add(Dense(512, init='glorot_normal', activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    model.add(Dropout(0.5))
    model.add(Dense(256, init='glorot_normal', activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    model.add(Dropout(0.5))
    model.add(Dense(output_dim, init='glorot_normal', activation='softmax', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-5)))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])   # 'categorical_crossentropy'
    return model


# def create_nn_model(input_dim, output_dim=4):

#     return model


In [0]:
def main():
    # start_time = time.time()

    # logging.basicConfig(level=logging.DEBUG,
    #                     format='[%(asctime)s]: %(message)s ',
    #                     datefmt='%Y-%m-%d %H:%M:%S',
    #                     stream=sys.stdout,
    #                     filemode="w"
    #                     )

    # load data
    logging.info('Load data')
    train_df, test_df = load_data(train_data_path='train.csv', test_data_path='test.csv')
    X, y, X_submission = process_data(train_df, test_df, transform='sqrt')

    # training phase 1
    logging.info('Training phase 1')
    models = []

    # KNN
    models += [KNeighborsClassifier(n_neighbors=2, n_jobs=-1),
               KNeighborsClassifier(n_neighbors=4, n_jobs=-1),
               KNeighborsClassifier(n_neighbors=8, n_jobs=-1),
               KNeighborsClassifier(n_neighbors=16, n_jobs=-1),
               KNeighborsClassifier(n_neighbors=32, n_jobs=-1),
               KNeighborsClassifier(n_neighbors=64, n_jobs=-1),
               KNeighborsClassifier(n_neighbors=128, n_jobs=-1),
               KNeighborsClassifier(n_neighbors=256, n_jobs=-1),
               KNeighborsClassifier(n_neighbors=512, n_jobs=-1),
               KNeighborsClassifier(n_neighbors=1024, n_jobs=-1)]

    # Logistic Regression
    models += [LogisticRegression(penalty='l1', solver='liblinear', n_jobs=-1, C=0.43, class_weight=None),
               LogisticRegression(penalty='l2', solver='lbfgs', n_jobs=-1, C=0.01, class_weight=None, multi_class='multinomial')]

    # Random Forest
    models += [RandomForestClassifier(criterion='gini', n_jobs=-1, n_estimators=250, max_features=18, min_samples_split=5, max_depth=46, class_weight=None)]

    # sklearn extra-trees classifier
    models += [ExtraTreesClassifier(criterion='gini', n_estimators=500, n_jobs=-1)]

    # xgboost
    models += [XGBClassifier(base_score=0.5, colsample_bylevel=1,
                             colsample_bytree=0.6, gamma=0,
                             learning_rate=0.03, max_delta_step=0, max_depth=28,
                             min_child_weight=6, n_estimators=2, nthread=-1,
                             objective='multi:softprob', reg_alpha=0, reg_lambda=1,
                             scale_pos_weight=1, seed=0, silent=True, subsample=0.8)]

    # NN
    models += [KerasClassifier(build_fn=create_2_layer_keras_model, input_dim=93, output_dim=9, nb_epoch=100, batch_size=256, verbose=0),
               KerasClassifier(build_fn=create_3_layer_keras_model, input_dim=93, output_dim=9, nb_epoch=100, batch_size=256, verbose=0)]

    models += [KerasClassifier(build_fn=create_2_layer_keras_model, input_dim=X.shape[1], output_dim=9, nb_epoch=100, batch_size=256, verbose=0)]

    logging.info('List of models to train and ensemble:')
    for model in models:
        print model
    num_models = len(models)

    train_models_pred, test_models_pred, summary = models_CV_train(models, X, y, X_submission, n_classes=9, n_folds=5)
    # save phase 1 intermediate prediction results
    # np.savetxt('train_models_pred.csv', train_models_pred)
    # np.savetxt('test_models_pred.csv', test_models_pred)

    logging.info('Summary for phase 1 model performance:')
    for i in sorted(summary.keys()):
        print 'Model %d: %f' % (i, summary[i])

    # training phase 2
    logging.info('Training phase 2: stacking model predictions from phase 1')
    stacking_models = []

    # stacking models' predictions using xgboost
    stacking_models +=[XGBClassifier(base_score=0.5, colsample_bylevel=1,
                                     colsample_bytree=0.6, gamma=0,
                                     learning_rate=0.03, max_delta_step=0, max_depth=90,
                                     min_child_weight=6, n_estimators=500, nthread=-1,
                                     objective='multi:softprob', reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=0, silent=True, subsample=0.8)]

    # stacking models' predictions using NN
    stacking_models += [KerasClassifier(build_fn=create_2_layer_keras_model, input_dim=num_models * 9, output_dim=9, nb_epoch=2, batch_size=256, verbose=0)]

    y_submission_pred = stacking_models_CV_train(stacking_models, train_models_pred, y, test_models_pred, n_classes=9, n_folds=5)
    y_submission_pred = y_submission_pred.mean(axis=2)

    # write submission
    logging.info("Write submission: averaging stacking models' predictions")
    timestamp = time.strftime("%Y-%m-%d_%H:%M:%S")
    columns = ['Class_' + str(i + 1) for i in range(9)]
    submission_df = pd.DataFrame(y_submission_pred, columns=columns)
    submission_df.index = submission_df.index + 1
    submission_df.to_csv('submission_%s.csv' % timestamp, sep=',', index_label='id')
    
    end_time = time.time()
    logging.info("Run complete: %s elapsed" % elapsed_time(start_time, end_time))

## Load data

`Highlights of the dataset`

```
Accident_ID:                unique id assigned to each row
Accident_Type_Code:           the type of accident (factor, not numeric)
Cabin_Temperature:            the last recorded temperature before the incident, measured in degrees fahrenheit
Turbulence_In_gforces:      the recorded/estimated turbulence experienced during the accident
Control_Metric:               an estimation of how much control the pilot had during the incident given the factors at play
Total_Safety_Complaints:     number of complaints from mechanics prior to the accident
Days_Since_Inspection:        how long the plane went without inspection before the incident
Safety_Score:                 a measure of how safe the plane was deemed to be
Severity:                    a description (4 level factor) on the severity of the crash [Target]
```

In [0]:
COLAB = True 
LOAD_DATASET = False

if COLAB:
  data_path = '/content/gdrive/My Drive/ML Projects/Hacker/data/'
  models_path = '/content/gdrive/My Drive/ML Projects/Hacker/models/'
  out_path = '/content/gdrive/My Drive/ML Projects/Hacker/'
else:
  data_path = '/home/ime/Documents/PyCharmProjects/AV/LTFS/data/'
  models_path = '/home/ime/Documents/PyCharmProjects/AV/LTFS/models/'
  out_path = '/home/ime/Documents/PycharmProjects/AV/LTFS/submissions/'

In [0]:
class_map = {
    'Minor_Damage_And_Injuries': 0,
    'Significant_Damage_And_Fatalities': 1,
    'Significant_Damage_And_Serious_Injuries': 2,
    'Highly_Fatal_And_Damaging': 3}

inverse_class_map = {
    0: 'Minor_Damage_And_Injuries',
    1: 'Significant_Damage_And_Fatalities',
    2: 'Significant_Damage_And_Serious_Injuries',
    3: 'Highly_Fatal_And_Damaging'}


In [0]:
def load_data(train_data_path=data_path+'train.csv', test_data_path=data_path+'test.csv'):

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    train_df['Severity'] = train_df['Severity'].map(class_map).astype(np.uint8)
        
    return train_df, test_df

In [16]:
# if LOAD_DATASET:
#     # load processed data NN
#     data_train = pd.read_csv(data_path+'train_dataset_ml_18f_v2.csv')
#     data_test = pd.read_csv(data_path+'test_dataset_ml_18f_v2.csv')
# else: 
#     # load raw data
#     data_train = pd.read_csv(data_path+'train.csv')
#     data_test = pd.read_csv(data_path+'test.csv')
#     # subm_file = pd.read_csv(data_path+'sample_submission.csv')

train_df, test_df = load_data()

print ("Shape of Training Data:{}".format(train_df.shape))
print ("Shape of Testing Data:{}".format(test_df.shape))

print(train_df['Severity'].value_counts(normalize=True))

Shape of Training Data:(10000, 12)
Shape of Testing Data:(2500, 11)
3    0.3049
2    0.2729
0    0.2527
1    0.1695
Name: Severity, dtype: float64


In [17]:
train_df.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,0,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,0,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,1,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,2,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,1,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


In [0]:
test_df.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,19.497717,16,6,72.151322,0.388959,78.32,4,37949.724386,2,0.069692,1
1,58.173516,15,3,64.585232,0.250841,78.60,7,30194.805567,2,0.002777,10
2,33.287671,15,3,64.721969,0.336669,86.96,6,17572.925484,1,0.004316,14
3,3.287671,21,5,66.362808,0.421775,80.86,3,40209.186341,2,0.199990,17
4,10.867580,18,2,56.107566,0.313228,79.22,2,35495.525408,2,0.483696,21


## Process data

In [0]:
for df in [train_df, test_df]:
    df = df.drop(['Accident_ID'], axis=1, inplace=True)

In [22]:
train_df.dtypes, test_df.dtypes

(Severity                     uint8
 Safety_Score               float64
 Days_Since_Inspection        int64
 Total_Safety_Complaints      int64
 Control_Metric             float64
 Turbulence_In_gforces      float64
 Cabin_Temperature          float64
 Accident_Type_Code           int64
 Max_Elevation              float64
 Violations                   int64
 Adverse_Weather_Metric     float64
 dtype: object, Safety_Score               float64
 Days_Since_Inspection        int64
 Total_Safety_Complaints      int64
 Control_Metric             float64
 Turbulence_In_gforces      float64
 Cabin_Temperature          float64
 Accident_Type_Code           int64
 Max_Elevation              float64
 Violations                   int64
 Adverse_Weather_Metric     float64
 dtype: object)

In [23]:
# select features
num_feats = [col for col in train_df.columns if train_df[col].dtype in ['float64'] ]

# select categorical features
categ_feats = ['Accident_Type_Code']

print('# num features: ', len(num_feats), len(train_df.columns))

# num features:  6 11


In [26]:
X, y, X_submission = process_data(train_df, test_df, 
                                                    ylabel='Severity', 
                                                    numerical_features = num_feats,
                                                    standarization=True, 
                                                    discretization=False, 
                                                    transform=None 
                                  )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


In [27]:
X.shape, y.shape, X_submission.shape

((10000, 10), (10000,), (2500, 10))

## Training phase 1

In [70]:
models = []

# # KNN
# models += [
#         #    KNeighborsClassifier(n_neighbors=2, n_jobs=-1),
#         #    KNeighborsClassifier(n_neighbors=4, n_jobs=-1, p=2),
#         #    KNeighborsClassifier(n_neighbors=8, n_jobs=-1, p=1),
#         #    KNeighborsClassifier(n_neighbors=16, n_jobs=-1, p=2),
#         #    KNeighborsClassifier(n_neighbors=32, n_jobs=-1),
#         #    KNeighborsClassifier(n_neighbors=64, n_jobs=-1, p=2),
#         #    KNeighborsClassifier(n_neighbors=128, n_jobs=-1, p=2),
#            KNeighborsClassifier(n_neighbors=256, n_jobs=-1, p=2),
#            KNeighborsClassifier(n_neighbors=512, n_jobs=-1, p=2),
#         #    KNeighborsClassifier(n_neighbors=1024, n_jobs=-1)
#            ]

# Logistic Regression
# models += [
#            LogisticRegression(penalty='l1', solver='liblinear', n_jobs=-1, C=0.1, class_weight=None),
#         #    LogisticRegression(penalty='l2', solver='lbfgs', n_jobs=-1, C=0.01, class_weight='balanced', multi_class='multinomial')
#            ]

# Random Forest
models += [
        RandomForestClassifier(criterion='gini', n_jobs=-1, n_estimators=300, 
                                  max_features='sqrt', min_samples_split=10, max_depth=46, 
                                  class_weight=None, bootstrap=False),
        RandomForestClassifier(criterion='gini', n_jobs=-1, n_estimators=700, 
                                  max_features='sqrt', max_depth=None, 
                                  class_weight='balanced')]


# Extra-trees classifier
# models += [ExtraTreesClassifier(criterion='gini', n_estimators=300, n_jobs=-1)]

# XGBoost
models += [XGBClassifier(base_score=0.5, colsample_bylevel=1,
                          subsample=0.9,
                             colsample_bytree=0.6, gamma=0,
                             learning_rate=0.03, max_delta_step=0, max_depth=7,
                             min_child_weight=6, n_estimators=1100, nthread=-1,
                             objective='multi:softprob', reg_alpha=1.0, reg_lambda=2.0),
            XGBClassifier(base_score=0.5, colsample_bylevel=1,colsample_bytree=0.8,
                          subsample=0.95, gamma=0, objective='multi:softprob',
                             learning_rate=0.03, max_delta_step=0, max_depth=7,
                             min_child_weight=6, n_estimators=900, nthread=-1,
                             reg_alpha=1.0, reg_lambda=1.5) ]


# XGB best params:
#  {'subsample': 0.9, 'n_estimators': 1300, 'max_depth': 7, 'learning_rate': 0.1}
# XGB best_score 0.956784


# LGBM 
# LGBM best params:
#  {'subsample': 0.4, 'num_leaves': 128, 'num_iterations': 700, 'max_depth': 9, 'learning_rate': 0.2, 'feature_fraction': 0.8}
# best_score 0.9669990951908287

# LGBM best params:
#  {'subsample': 0.9, 'num_leaves': 16, 'num_iterations': 800, 'max_depth': 9, 'learning_rate': 0.1, 'feature_fraction': 0.9}
# LGBM best_score 0.956765

lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 4,
    'eval_metric': {'multi_logloss'},     # 'multi_logloss',    # 'multi_logloss' 'auc',  'f1', 'f1_macro', 'f1_micro', 
    'subsample': 0.4,
    # 'subsample_freq': 1,
    'learning_rate': 0.2,
    'num_iterations': 700,
    'num_leaves': 128,
    'feature_fraction': 0.8,
    'max_depth': 9,
    'lambda_l1': 1.5,
    'lambda_l2': 2.5 }


models += [ lgb.LGBMClassifier(**lgb_params) ]


# NN
# models += [
#            KerasClassifier(build_fn=create_2_layer_keras_model, input_dim=X.shape[1], output_dim=4, nb_epoch=100, batch_size=256, verbose=0),
#            KerasClassifier(build_fn=create_3_layer_keras_model, input_dim=X.shape[1], output_dim=4, nb_epoch=100, batch_size=256, verbose=0)]

# models += [KerasClassifier(build_fn=create_2_layer_keras_model, input_dim=X.shape[1], output_dim=4, nb_epoch=100, batch_size=256, verbose=0)]




print('List of models to train and ensemble:')
for model in models:
    print(model)
    num_models = len(models)
print('# base models:', num_models)

List of models to train and ensemble:
RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=46, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_esti

In [73]:
for i in models:
    print(type(i))

# type(models[2]) == XGBClassifier   

# models[5].fit()

# xgb.XGBClassifier().fit(X, )

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
<class 'xgboost.sklearn.XGBClassifier'>
<class 'xgboost.sklearn.XGBClassifier'>
<class 'lightgbm.sklearn.LGBMClassifier'>


In [74]:
start_time1 = time.time()

train_models_pred, test_models_pred, summary = models_CV_train(models, X, y, X_submission, n_classes=4, n_folds=5)

print('='*10)
print('Training phase 1 elapsed time:', start_time1 - time.time(), '[sec]')

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Model 0: RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=46, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
  Fold 0
logloss: 0.311254
  Fold 1
logloss: 0.307694
  Fold 2
logloss: 0.297337
  Fold 3
logloss: 0.298472
  Fold 4
logloss: 0.292852
model average logloss: 0.301522
Model 1: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, m

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


logloss: 0.151244
  Fold 1


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

logloss: 0.148249
  Fold 3


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


logloss: 0.140030
  Fold 4


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


logloss: 0.114747
model average logloss: 0.140446


In [0]:
# save phase 1 intermediate prediction results
np.savetxt(data_path+'train_models_pred_3.csv', train_models_pred)
np.savetxt(data_path+'test_models_pred_3.csv', test_models_pred)

In [76]:
train_models_pred.shape, test_models_pred.shape

((10000, 20), (2500, 20))

In [0]:
with open(models_path+'base_models_3.pickle', 'wb') as f:
    pickle.dump(obj=models, file=f)

# RandomForestClassifier   ->
# RandomForestClassifier   ->
# XGBClassifier'   ->
# XGBClassifier'   ->
# LGBM             -> 

In [84]:
print('Summary for phase 1 model performance:')
score_1 = 0
for i in sorted(summary.keys()):
    print('Model %d %s: %f' % (i, type(models[i]), summary[i]))
    score_1 += summary[i]
print('Avg score:', score_1/len(models))

# Model 0 : 1.370841   KNN
# Model 1 : 1.368519   KNN
# Model 2 : 1.122398   LR
# Model 3 : 0.330810   RF
# Model 4 : 0.350650   RF
# Model 5 : 0.505754   ET
# Model 6 : 0.142307   XGB


Summary for phase 1 model performance:
Model 0 <class 'sklearn.ensemble._forest.RandomForestClassifier'>: 0.301522
Model 1 <class 'sklearn.ensemble._forest.RandomForestClassifier'>: 0.317112
Model 2 <class 'xgboost.sklearn.XGBClassifier'>: 0.139199
Model 3 <class 'xgboost.sklearn.XGBClassifier'>: 0.133411
Model 4 <class 'lightgbm.sklearn.LGBMClassifier'>: 0.140446
Avg score: 0.20633809638058698


## Training phase 2

In [81]:
# training phase 2
print('Training phase 2: stacking model predictions from phase 1')
stacking_models = []

# stacking models' predictions using xgboost

xgb_params = dict(objective='multi:softprob', 
              n_estimators=300,
              colsample_bylevel=1, 
              subsample=0.9,
              colsample_bytree=0.8, 
              gamma=0, 
              learning_rate=0.03, 
              reg_alpha=0, reg_lambda=1.5)

stacking_models +=[XGBClassifier(**xgb_params)]



# # stacking models' predictions using NN
# stacking_models += [
#                     KerasClassifier(build_fn=create_2_layer_keras_model, 
#                                     input_dim=num_models * 4, 
#                                     output_dim=4, 
#                                     nb_epoch=5, 
#                                     batch_size=256, 
#                                     verbose=0)
#                     ]

# Tune a NN with stacked features ...

# stacking_models += [ build_nn_model() ]



Training phase 2: stacking model predictions from phase 1


In [82]:
y_submission_pred = stacking_models_CV_train(stacking_models, 
                                             train_models_pred, 
                                             y, 
                                             test_models_pred, 
                                             n_classes=4, 
                                             n_folds=5)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Stacking Model 0 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.03, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1.5, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)
Fold 0
  logloss: 0.141453
Fold 1
  logloss: 0.141030
Fold 2
  logloss: 0.134920
Fold 3
  logloss: 0.132865
Fold 4
  logloss: 0.101952
model average logloss: 0.130444


In [0]:
# with open(models_path+'stacking_models.pickle', 'wb') as f:
#     pickle.dump(obj=stacking_models, file=f) 

## Avg. predictions

In [85]:
y_submission_pred.shape

(2500, 4, 1)

In [0]:
y_submission_pred = y_submission_pred.mean(axis=2)

In [87]:
y_submission_pred

array([[3.61541670e-04, 1.75162402e-04, 2.22306215e-04, 9.99240983e-01],
       [4.51654120e-04, 9.98184347e-01, 3.97955539e-04, 9.66024504e-04],
       [5.58877632e-04, 3.04629136e-04, 9.98546314e-01, 5.90222736e-04],
       ...,
       [5.40982315e-04, 2.67280260e-04, 9.98700869e-01, 4.90859873e-04],
       [4.93156066e-04, 2.75574281e-04, 9.98663795e-01, 5.67445881e-04],
       [2.92838548e-04, 1.73205341e-04, 4.52458195e-04, 9.99081492e-01]])

In [88]:
test_preds_classes = np.argmax(y_submission_pred, axis=1)

test_preds_classes.shape

(2500,)

## Submission

In [89]:
# write submission
print("Write submission: averaging stacking models' predictions")

test_ids = pd.read_csv(data_path+'test.csv')['Accident_ID']

submission = pd.DataFrame({'Accident_ID': test_ids,    #data_test['Accident_ID'], 
                           'Severity': np.vectorize(inverse_class_map.get)(test_preds_classes)})

submission.to_csv(out_path+'submission_stack2_v2_meta_xgb_3.csv', index=False)

Write submission: averaging stacking models' predictions


In [90]:
submission.head()

,Accident_ID,Severity
0,1,Highly_Fatal_And_Damaging
1,10,Significant_Damage_And_Fatalities
2,14,Significant_Damage_And_Serious_Injuries
3,17,Highly_Fatal_And_Damaging
4,21,Significant_Damage_And_Fatalities


In [91]:
submission.Severity.value_counts(normalize=True)

Highly_Fatal_And_Damaging                  0.3044
Minor_Damage_And_Injuries                  0.2748
Significant_Damage_And_Serious_Injuries    0.2560
Significant_Damage_And_Fatalities          0.1648
Name: Severity, dtype: float64

## Test saved model for reproducibility

In [0]:
# # # load LGBM model
# with open(models_path+'lgbm2_22f_v1_model.pickle', 'rb') as f:
#     lgb_model = pickle.load(f) 

In [0]:
# test_data = data_test[feats].values

# test_preds_classes = lgb_model.predict(test_data, num_iteration=lgb_model.best_iteration_)
# print(test_preds_classes.shape)

In [0]:
#  test_ids = pd.read_csv(data_path+'test.csv')['Accident_ID']

# submission = pd.DataFrame({'Accident_ID': test_ids,      # data_test['Accident_ID'], 
#                            'Severity': np.vectorize(inverse_class_map.get)(test_preds_classes)})

# submission.to_csv(out_path+'submission_test_lgbm_v1_10f.csv', index=False)

In [0]:
# submission.head()